# Sentiment Analysis with an RNN

The Sentiment Analysis model is implement using a recurrent neural network specifically LSTM (Long Short-Term Memory). 

>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. 

### Network Architecture

The architecture for this network is shown below.

<img src= "https://drive.google.com/uc?id=1XN5-ODOUQIwX0TYHNcPnHt5a0izrHsnS" width=40%>

>**First, the words are passed to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. Instead of using a pre-trained Word2Vec model, it is good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and gives the ability to include information about the *sequence* of words in the movie review data. 

>**Finally, the LSTM outputs will go to a sigmoid output layer.** Here a sigmoid function is used because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1. 

The only sigmoid output which is interesting for this model is the **very last one**; ignore the rest. Then the loss is calculated by comparing the output at the last time step and the training label (pos or neg).

In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from string import punctuation
from collections import Counter

In [2]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


## Data pre-processing

The first step when building a neural network model is getting the data into the proper form to feed into the network. Since the embedding layers are used, each word needs to be encoded with an integer.

Here the following processing steps are carried out:
>* Gettoing rid of periods and extraneous punctuation.
* Also, In the data used here, the reviews are delimited with newline characters `\n`. To deal with those, the text is split into each review using `\n` as the delimiter. 
* Then combine all the reviews back together into one big string.
* Then got all the text without the newlines and split it into individual words.

In [3]:
def preprocessing(reviews):
    
    # get rid of punctuation
    reviews = reviews.lower() # lowercase, standardize
    all_text = ''.join([c for c in reviews if c not in punctuation])

    # split by new lines and spaces
    reviews_split = all_text.split('\n')
    all_text = ' '.join(reviews_split)

    # create a list of words
    words = all_text.split()
    
    return words, reviews_split

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then convert each review into integers so they can be passed into the network.

* For the purpose of padding the input vectors with zeros (which is explained in the latter section), make sure the integers **start at 1, not 0**.
* Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [4]:
def EncodingWords(words, reviews_split):
    ## Build a dictionary that maps words to integers
    word_count = Counter(words)
    vocab = sorted(word_count, key=word_count.get, reverse=True)
    vocab_to_int = {word: pos for pos,word in enumerate(vocab, 1)}

    ## use the dict to tokenize each review in reviews_split
    ## store the tokenized reviews in reviews_ints
    reviews_ints = []
    for review in reviews_split:
        reviews_ints.append([vocab_to_int[word] for word in review.split()])
        
    return vocab_to_int, reviews_ints

### Encoding the labels

Similar to encoding the words, the labels namely, "positive" and "negative" are converted to 0 and 1 so that they could be used in our network.

* Converted labels from `positive` and `negative` to 1 and 0, respectively, and place those in a new list, `encoded_labels`.

In [5]:
# 1=positive, 0=negative label conversion
def EncodingLabels(labels):
    labels_split = labels.split('\n')
    encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])
    
    return encoded_labels

### Removing Outliers

As an additional pre-processing step, make sure that the reviews are in good shape for standard processing. That is, the network will expect a standard input text size, and so, the reviews are shaped into a specific length. There are two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that the reviews will be of the same length.

In [6]:
def RemoveEmptyReviews(encoded_labels, reviews_ints):
    ## remove any reviews/labels with zero length from the reviews_ints list.
    non_zero_index = [index for index, review in enumerate(reviews_ints) if len(review)!=0]

    reviews_ints = [reviews_ints[i] for i in non_zero_index]
    encoded_labels = np.array([encoded_labels[i] for i in non_zero_index])
    
    return encoded_labels, reviews_ints

---
## Padding sequences

To deal with both short and very long reviews, all the reviews are either padded or truncated to a specific length. For reviews shorter than some `seq_length`, pad with 0s. For reviews longer than `seq_length`, truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> The following function returns an array `features` that contains the padded data, of a standard size, which is then passed to the network. 
* The data should come from `review_ints`, since the integers are fed to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.


**The final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

In [7]:
def pad_trunc_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## implement function

    features= [review[:seq_length] if (len(review) >= seq_length) else ([0]*(seq_length-len(review))+review) for review in reviews_ints]
    features = np.array(features).reshape(len(reviews_ints), seq_length)
    
    return features

---
## Data splitting, DataLoaders and Batching

Splitting the dataset into training, validation and test set is necessary to estimate the performance of the trained model on a dataset which is never seen by the model. In this case, a fraction of dataset is reserved as a 'training' set and the remaining data is split into half thus resulting in 'validation' and 'test' set.

After creating training, validation, and test data, the DataLoaders are created for the data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

This is an alternative to creating a generator function for batching our data into full batches.

In [8]:
def train_val_test_loader(features, encoded_labels, train_frac, batch_size):

    ## split data into training, validation, and test data (features and labels, x and y)
    train_idx = int(train_frac*len(features))
    train_x, remain_x = features[:train_idx], features[train_idx:]
    train_y, remain_y = encoded_labels[:train_idx], encoded_labels[train_idx:]

    val_test_idx = int(0.5*len(remain_x))
    val_x, test_x = remain_x[:val_test_idx], remain_x[val_test_idx:]
    val_y, test_y = remain_y[:val_test_idx], remain_y[val_test_idx:]
    
    ## printing the shapes of your resultant feature data
    print('Training set shape: {}'.format(train_x.shape))
    print('Validation set shape: {}'.format(val_x.shape))
    print('Test set shape: {}'.format(test_x.shape))
    
    # creating Tensor datasets
    train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
    test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

    # dataloaders
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


    return train_loader, valid_loader, test_loader

---
# Sentiment Network with PyTorch

Below is the definition of the network.

<img src= "https://drive.google.com/uc?id=1XN5-ODOUQIwX0TYHNcPnHt5a0izrHsnS" width=40%>

The layers are as follows:
1. An embedding layer that converts our word tokens (integers) into embeddings of a specific size.
2. An LSTM layer defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

An embedding layer is needed, because there are 74000+ words in the vocabulary for the dataset used. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, here an embedding layer is used and which is used as a lookup table. It is also possible to train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

An LSTM is used in this recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, the network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships. 

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [27]:
class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.b
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.output = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_words, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = input_words.size(0)
        
        input_words = input_words.long()
        embed_out = self.embed(input_words)
        lstm_out, hidden = self.lstm(embed_out, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        sig_out = self.sigmoid(self.output(lstm_out))
        sig_out = sig_out.view(batch_size, -1)
        sig_last = sig_out[:,-1]
        
        # return last sigmoid output and hidden state
        return sig_last, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

---
## Training

In [10]:
def training_validation(net, train_loader, valid_loader, batch_size, epochs, train_on_gpu, 
                        criterion, optimizer, print_every = 100, clip=5):
    
    """ 
    This function trains and validates the LSTM model with the train and validation loader.
    
    Here a special kind of entropy loss is used, which is designed to work with a 
    single Sigmoid output called "Binary Cross Entropy Loss", applies cross entropy loss to a single value between 0 and 1.
    """
    # training params 
    counter = 0

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    
    # move model to GPU, if available
    if(train_on_gpu):
        net.cuda()

    net.train()
    # train for some number of epochs
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        # batch loop
        for inputs, labels in train_loader:
            counter += 1
            train_losses = []
            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            train_losses.append(loss.item())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for inputs, labels in valid_loader:

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    if(train_on_gpu):
                        inputs, labels = inputs.cuda(), labels.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output.squeeze(), labels.float())

                    val_losses.append(val_loss.item())

                if np.mean(val_losses) <= valid_loss_min:
#                 print('Validation loss decreased. Saving model ...')
                  torch.save(net.state_dict(), 'sentiment_analysis.pt')
                  valid_loss_min = np.mean(val_losses)

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(np.mean(train_losses)),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))
    return net    


---
## Testing


In [11]:
def testing(net, test_loader, criterion, batch_size):
    """
    This funtion tests the performance of the trained network with the test data for which the labels already exists
    """
    # Get test data loss and accuracy

    test_losses = [] # track loss
    num_correct = 0

    # init hidden state
    h = net.init_hidden(batch_size)

    net.eval()
    # iterate over test data
    for inputs, labels in test_loader:

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # get predicted outputs
        output, h = net(inputs, h)

        # calculate loss
        test_loss = criterion(output.squeeze(), labels.float())
        test_losses.append(test_loss.item())

        # convert output probabilities to predicted class (0 or 1)
        pred = torch.round(output.squeeze())  # rounds to the nearest integer

        # compare predictions to true label
        correct_tensor = pred.eq(labels.float().view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        num_correct += np.sum(correct)

    # accuracy over all test data
    test_acc = num_correct/len(test_loader.dataset)
    
    return test_losses, test_acc

# Sentiment Analysis

### Loading data

In [13]:
# read data from text files
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()
    
print(reviews[:1000])
print()
print(labels[:18])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

### Data pre-processing

In [14]:
words, reviews_split = preprocessing(reviews)

vocab_to_int, reviews_ints = EncodingWords(words, reviews_split)

# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))
print()
# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

encoded_labels = EncodingLabels(labels)

# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]
Zero-length reviews: 1
Maximum review length: 2514


In [15]:
if review_lens[0]>0:
    
    """
    If there are any reviews with zero length, then the following steps will remove those reviews. 
    This process thus removes outliers if any and allow the model to train more efficiently.
    
    Removing any reviews with zero length from the 'reviews_ints' list and their corresponding label in 'encoded_labels'.
    """
    print('Number of reviews before removing outliers: ', len(reviews_ints))
    encoded_labels, reviews_ints = RemoveEmptyReviews(encoded_labels, reviews_ints)
    print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


In [16]:
# Padding and truncating the reviews to match a specific sequence_length

"""
If the maximum review length is way too many steps for our RNN, then the following steps will truncate 
any super long reviews. This removes outliers and should allow the model to train more efficiently.

Removing any reviews with zero length from the 'reviews_ints' list and their corresponding label in 'encoded_labels'.
"""
seq_length = 200

features = pad_trunc_features(reviews_ints, seq_length)

## test statements 
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

# Splitting the dataset

In [17]:
train_frac=0.80   # The fraction of the data which will be alloted for training the data. Remaining data will be split into half for validation and test set.
batch_size = 50
train_loader, valid_loader, test_loader = train_val_test_loader(features, encoded_labels, train_frac, batch_size)

# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Training set shape: (20000, 200)
Validation set shape: (2500, 200)
Test set shape: (2500, 200)
Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[   0,    0,    0,  ...,  710,    2,  583],
        [   0,    0,    0,  ...,   12,  165, 2581],
        [   0,    0,    0,  ...,   18,    7,    7],
        ...,
        [   0,    0,    0,  ...,   85, 4082,  222],
        [1378, 1011,    7,  ...,    9,   41, 1580],
        [   0,    0,    0,  ...,   57,  148,    8]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 0])


# Training, Validating and Testing the LSTM model

In [30]:
# Instantiate the model w/ hyperparams

vocab_size = len(vocab_to_int)+1   #Size of our vocabulary or the range of values for our input, word tokens.
output_size = 1    # Size of our desired output; the number of class scores we want to output (pos/neg). 
embedding_dim = 300 # Number of columns in the embedding lookup table; size of our embeddings.
hidden_dim = 256   # Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
n_layers = 3   # Number of LSTM layers in the network. Typically between 1-3
lr=0.00015 # Learning rate for the optimizer.
epochs = 5   # Number of times to iterate through the training dataset.
clip = 5   # The maximum gradient value to clip at (to prevent exploding gradients).
print_every = 200   # Print the training and validation loss for every particular number of steps
drop_prob = 0.45

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=drop_prob)

print('The Network architecture of Sentiment Analysis model is: \n\n', net)

# loss and optimization functions

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

print('\n\nTraining and validation has begun: \n\n')
net = training_validation(net, train_loader, valid_loader, batch_size, epochs, train_on_gpu, 
                          criterion, optimizer, print_every, clip)

print('\n\nTesting has begun: \n\n')
test_losses, test_acc = testing(net, test_loader, criterion, batch_size)

print("\t Test loss: {:.3f}".format(np.mean(test_losses)))
print("\t Test accuracy: {:.3f}".format(test_acc))

The Network architecture of Sentiment Analysis model is: 

 SentimentRNN(
  (embed): Embedding(74073, 300)
  (lstm): LSTM(300, 256, num_layers=3, batch_first=True, dropout=0.45)
  (output): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


Training and validation has begun: 


Epoch: 1/5... Step: 200... Loss: 0.547504... Val Loss: 0.586342
Epoch: 1/5... Step: 400... Loss: 0.599924... Val Loss: 0.527879
Epoch: 2/5... Step: 600... Loss: 0.613822... Val Loss: 0.558392
Epoch: 2/5... Step: 800... Loss: 0.374181... Val Loss: 0.452838
Epoch: 3/5... Step: 1000... Loss: 0.337049... Val Loss: 0.486345
Epoch: 3/5... Step: 1200... Loss: 0.304758... Val Loss: 0.439330
Epoch: 4/5... Step: 1400... Loss: 0.206904... Val Loss: 0.471228
Epoch: 4/5... Step: 1600... Loss: 0.387008... Val Loss: 0.439708
Epoch: 5/5... Step: 1800... Loss: 0.230577... Val Loss: 0.464881
Epoch: 5/5... Step: 2000... Loss: 0.336348... Val Loss: 0.456677


Testing has begun: 


	 Test loss: 0.472
	 Tes

### Inference on a test review

To check if the trained model predicts the correct type of review for any given text. The text used for inference was never used for training, validating or testing the model.


In [19]:
def infer_preprocessing(reviews, sequence_length):
    reviews = reviews.lower() # lowercase, standardize
    all_text = ''.join([c for c in reviews if c not in punctuation])

    # create a list of words
    words = all_text.split()
    
    ## Build a dictionary that maps words to integers
    word_count = Counter(words)
    vocab = sorted(word_count, key=word_count.get, reverse=True)
    vocab_to_int = {word: pos for pos,word in enumerate(vocab, 1)}

    ## use the dict to tokenize each review in reviews_split
    ## store the tokenized reviews in reviews_ints
    reviews_ints = []
    reviews_ints.append([vocab_to_int[word] for word in words])
        
    # Trim the review according to the sequence length
    features = pad_trunc_features(reviews_ints, seq_length=sequence_length)
    
    return features

In [20]:
def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    
    
    # print custom response based on whether test_review is pos/neg
    net.eval()
    
    features = infer_preprocessing(test_review, sequence_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    batch_size = feature_tensor.size(0)
    
    # init hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get predicted outputs
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())
    
    # print custom response
    if(pred.item()==1):
        return "Positive review. \t Prediction value, pre-rounding: {:.6f}".format(output.item())
    else:
        return "Negative review. \t Prediction value, pre-rounding: {:.6f}".format(output.item())

In [21]:
# List of reviews for testing
seq_length=200

review1 = 'This movie had the best acting and the dialogue was so good. I loved it.'
review2 = 'The worst movie I have seen; acting was terrible and I want my money back.'
review3 = 'a big, brashly beautiful, grandiosely enjoyable one'
review4 = 'the story isn’t very interesting or original , the plot is formulaic, the characters are boring and stereotypical '
review5 = 'a great movie,  appealing to audience of every age.'

In [32]:
print('Ground truth: Positive review. Prediction: ', predict(net, review1, seq_length))
print('\nGround truth: Negative review. Prediction: ', predict(net, review2, seq_length))
print('\nGround truth: Positive review. Prediction: ', predict(net, review3, seq_length))
print('\nGround truth: Negative review. Prediction: ', predict(net, review4, seq_length))
print('\nGround truth: Positive review. Prediction: ', predict(net, review5, seq_length))

Ground truth: Positive review. Prediction:  Positive review. 	 Prediction value, pre-rounding: 0.520384

Ground truth: Negative review. Prediction:  Negative review. 	 Prediction value, pre-rounding: 0.475639

Ground truth: Positive review. Prediction:  Positive review. 	 Prediction value, pre-rounding: 0.710473

Ground truth: Negative review. Prediction:  Negative review. 	 Prediction value, pre-rounding: 0.498521

Ground truth: Positive review. Prediction:  Positive review. 	 Prediction value, pre-rounding: 0.724536
